<a href="https://colab.research.google.com/github/raqueeb/TensorFlow2/blob/master/embedding_bangla_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## এমবেডিং, ওয়ার্ড এমবেডিং এবং বাংলায় টেক্সট অ্যানালাইসিস


একটা জিনিস ভেবে দেখেছেন কি? আমরা এ পর্যন্ত মেশিন লার্নিং মডেলের ইনপুট হিসেবে যা দিয়েছি তা সবই সংখ্যা। মনে করে দেখুন, এপর্যন্ত সব মডেলের ক্লাসিফিকেশন অথবা রিগ্রেশন এর জন্য যা দিয়েছি সব সংখ্যায় দিয়েছি। তার পাশাপাশি ইমেজ নিয়ে যখন কাজ করেছি তখনো কিন্তু ইমেজ (সেটা গ্রেস্কেল হোক আর কালার হোক - তার জন্য গ্রেস্কেল ইনটেনসিটি অথবা কালারের আরজিবি চ্যানেলের আউটপুট), সবকিছুই সংখ্যায় গিয়েছে। এর অর্থ হচ্ছে মেশিন লার্নিং/ডিপ লার্নিং মডেল সংখ্যা ছাড়া আর কিছু বোঝেনা। আর বুঝবেই বা কিভাবে? সেতো যন্ত্র। আর মানুষের সবকিছুই কমপ্লেক্স। 

সেদিক থেকে মানুষের ভাষা আরো অনেক কমপ্লেক্স। আমরা একেকজন একেক ভাষায় কথা বলি, ভাষাগুলোর মধ্যে সংযোগ/সিমিলারিটি এবং কি বলতে গিয়ে কি বলে ফেললাম এবং তার ফলাফল, তার পাশাপাশি অনেক শব্দ একটা ভাষায় যা বোঝায় সেটা অন্য ভাষায় তার বৈপরীত্য দেখায়। এখন আপনি বাংলায় কথা বললেও সেটার মধ্যে ৪০% বাইরের শব্দ ব্যবহার করলে তো আরো সমস্যা। আর এই কারণে টেক্সট নিয়ে কাজ করা বেশ কমপ্লেক্স। 

যেকোনো ল্যাঙ্গুয়েজে তার প্রতিটা শব্দের একটা অর্থ আছে। তবে এটার অর্থ অনেক সময় নির্ভর করে কনটেক্সটে বা শব্দটা বাক্যের মধ্যে কোথায় এই মুহূর্তে আছে। একই শব্দের আবার অনেকগুলো কনটেক্সচুয়াল অর্থ থাকে সে কারণে শব্দকে শুধুমাত্র শব্দ বা অক্ষর লেভেলে কাজ করলে হবে না। কারণ, ডিপ লার্নিং মডেল যেহেতু সংখ্যা ছাড়া কিছু বোঝেনা, সে কারণে একেক ভাষার একেক বুলি এবং বকাবকি এর পাশাপাশি সেই ভাষাগুলোকে ঠিকমতো সংখ্যায় ট্রান্সফার করা একটা চ্যালেঞ্জ এর কাজ অবশ্যই।

## কেন টাইম সিরিজ নিয়ে আলাপ হয়নি?

এই বইতে আমি ইচ্ছে করে ‘টাইম সিরিজ’ যোগ করিনি, কারণ সেটার এপ্লিকেশন লেভেল এখনো বেসিক লেভেলে নেই। তবে, এই রিকারেন্ট নিউরাল নেটওয়ার্ককে শিখিয়ে দিলে সে (আরএনএন) ফ্রি ফর্ম (ইচ্ছেমতো) টেক্সট জেনারেট করতে পারে। আমরা যেমন দেখেছি ‘এল এস টি এম’, (লঙ শর্ট টার্ম মেমোরি) নেটওয়ার্কে ‘শেক্সপিয়ার’ ক্লাসিক পড়তে দিলে, সে শেক্সপিয়ারের মতো আরেকটা ক্লাসিক লিখে ফেলেছে, যেখানে এই নেটওয়ার্কের মধ্যে শব্দ, বাক্য এবং ব্যাকরণ তৈরির কোন ধারণা নেই। কারণ, ডিপ লার্নিং প্রচুর ক্লাসিক বই পড়ে বুঝেছে কিভাবে শব্দ, বাক্য বা তার ব্যাকরণ ব্যবহার করতে হয় এর ভেতরে না ঢুকেই। টাইম সিরিজের ব্যাপারটা হচ্ছে সে পরের জিনিসটা প্রেডিক্ট করবে। আগের সময়ে কি ছিলো, সেটাকে ধরে এরপরে কি কি আসতে পারে সেটাই বলবে সে। না বুঝে। যদি শব্দ লেভেলে দেখি, ‘আমি’ এর পর কি আসতে পারে তাহলে ‘ভালো’ আসতে পারে, কারণ ‘আমি ভালো আছি’ একটা বহুল প্রচলিত বাক্য। 

বড় ব্যাপার হচ্ছে এই অক্ষর থেকেই ‘এল এস টি এম’ আস্তে আস্তে মানবিক ব্যাকরন এবং কিভাবে একটা বাক্য তৈরি করতে হয় সে ধরনের একটা ধারণা পেয়ে থাকে। এটা সে কোন কিছু বুঝে করে না। প্যাটার্ন থেকে করে। আর সে কারণেই এটার উপরে আমরা খুব একটা ভরসা করব না। আমরা চাইব মেশিনকে শেখাতে, যেভাবে মানুষ ভাষা, শব্দ, ব্যাকরণ শেখে। ওই একই কারণে অক্ষর লেভেলে টেক্সট জেনারেশন নিয়ে আমরা এই মুহূর্তে আলাপ করব না। ইন্টারনেটে দেখতে পারেন কিভাবে একেকটা ‘এল এস টি এম’ নেটওয়ার্ক শেক্সপিয়ারের মত বড় বড় নাটক লিখে ফেলছে। এটা আসলে সে একটা ক্লাসিক্যাল লেখার প্যাটার্ন দেখে তার পার্সপেক্টিভ থেকে লিখেছে। বুঝে লেখেনি। 

## ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং

সেজন্য এর মধ্যে এসে যোগ হয়েছে ‘ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং’। ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং এর মধ্যে লিঙ্গুইস্টিকস, কম্পিউটার সায়েন্স, ইনফরমেশন ইঞ্জিনিয়ারিং এবং কৃত্রিম বুদ্ধিমত্তা ব্যাপারগুলো চলে এসেছে কাজের স্বার্থে। এমনিতেই মানুষ এবং যন্ত্রের মধ্যে একটা যোগসূত্র স্থাপন করা বেশ ঝামেলার ব্যাপার। ‘স্পিচ রিকগনিশন’ এর পাশাপাশি স্বয়ংক্রিয়ভাবে একটা ভাষা বুঝতে পারা এবং তার পাশাপাশি সেই ভাষায় বুঝে টেক্সট জেনারেট করা সহজ ব্যাপার নয়, যখন সবকিছুর পেছনে কাজ করে সংখ্যা। সবচেয়ে বড় কথা হচ্ছে আপনি একটা যন্ত্রকে শেখাচ্ছেন সে আপনার সাথে ন্যাচারালি যোগসুত্র স্থাপন করতে পারে। এই ‘ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং’ এর শুরুটা হচ্ছে সেই টেক্সটকে ঠিকমতো পড়তে পারা, সেটাকে নিজের মতো করে বোঝা এবং মানুষ যেভাবে একটা বাক্যের ‘ইনটেন্ট’ বুঝতে পারে সেভাবে তাকে বুঝিয়ে তার কাছ থেকে উত্তর বের করা। সহজ ব্যাপার নয়। তবে শুরু করতে হবে কোথাও। 

যেহেতু ‘ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং’ একটা বিশাল সাবজেক্ট, আমি এই বইতে ব্যাপারটা আনার ব্যর্থ চেষ্টা করব না। তবে কম্পিউটার কিভাবে ‘টেক্সট’ এর সাথে ‘ইন্টারঅ্যাক্ট’ করে সেটা নিয়েই দেখব আমরা সামনের চ্যাপ্টারগুলোতে। আমরা জানি মেশিন লার্নিং মডেল ভেক্টরকে ইনপুট হিসেবে নেয়। আর এই ভেক্টরগুলো হচ্ছে সংখ্যার অ্যারে। যখন আমরা টেক্সট মানে শব্দ এবং বাক্য নিয়ে আলাপ করব, তখন আমাদের প্রথম কাজ হবে এই স্ট্রিংগুলোকে কিভাবে সংখ্যায় পাল্টানো যায়। অর্থাৎ সেই বাক্য বা শব্দটি কিভাবে ‘ভেক্টরাইজ’ করা যায় মডেলে দেবার আগে। আমাদের এই চ্যাপ্টার থেকে শুরু করব কিভাবে আস্তে আস্তে ডিপ নিউরাল নেটওয়ার্ক দিয়ে বাংলার একটা অ্যাপ্লিকেশনে যাওয়া যায়।

In [1]:
try:
  # শুধুমাত্র টেন্সর-ফ্লো ২.x ব্যবহার করবো 
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
keras = tf.keras

TensorFlow 2.x selected.


আমরা যেহেতু ডিপ নিউরাল নেটওয়ার্কের কথা বলছি সেখানে সবকিছুতেই লেয়ার কনসেপ্ট কাজ করে। আজকে এখানে আমরা নতুন একটা লেয়ার নিয়ে কথা বলবো যেটাকে আমরা বলছি এম্বেডিং লেয়ার। শুরুতেই অনেকে বলবেন এম্বেডিং মানে কি? এর সাথে সংখ্যার সম্পর্ক কি?

In [0]:
from tensorflow.keras import layers
embedding_layer = layers.Embedding(1000, 5)

In [5]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[-0.0339667 , -0.04795448, -0.04234529,  0.02007512, -0.04314665],
       [-0.00510416,  0.04672292,  0.0373845 , -0.0407817 , -0.03552971],
       [ 0.03556449,  0.04361344,  0.04444368, -0.02098991, -0.03613399]],
      dtype=float32)

In [0]:
# অথবা টেন্সর-ফ্লো দিয়ে টোকেনাইজ করে দেখি

from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
tokenizer = Tokenizer()

text = ['আমি এখন বই পড়ি']
# text = tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

In [8]:
sequences

[[]]

## ‘এম্বেডিং’ সিমিলারিটি, একটা থেকে আরেকটা শব্দের দূরত্ব

মেশিন লার্নিং এর ভাষায় ‘এম্বেডিং’ হচ্ছে সিমিলারিটি, যদি শব্দের কথা বলি তাহলে একটা শব্দ থেকে আরেকটা শব্দ কতটুকু যুক্ত বা একটা শব্দ থেকে আরেকটা শব্দ কত দূরে? তাদের মধ্যে সম্পর্ক আছে কিনা? যেমন, রাজা’র সাথে ‘রানী’ শব্দটা কিন্তু সম্পর্কযুক্ত। যেমন, ‘মা’ শব্দের সাথে ‘বাবা’ সম্পর্কযুক্ত। ‘বাংলাদেশ’ শব্দের সাথে ‘ঢাকা’ সম্পর্কযুক্ত। সেই থেকে ‘আকাশ’ শব্দের সাথে ‘টেবিল’ কিন্তু বহু দূরে মানে তাদের মধ্যে সম্পর্ক টানা বেশ কঠিন, যদি না কোনদিন ‘আকাশ’ থেকে ‘টেবিল’ পড়ে।

এই এম্বেডিং ব্যাপারটা এসেছে কিছুটা ‘ওয়ান হট’ এনকোডিং থেকে। তবে, ব্যাপারটা ঠিক সেরকম নয়। শব্দকে যেহেতু আমাদেরকে সংখ্যায় পাল্টাতে হবে তাহলে আর বাকি উপায় কি? মনে আছে, আমাদের এই ‘ওয়ান হট’ এনকোডিং বা ‘ডামি ভেরিয়েবল’ ব্যাপারটা এসেছে শব্দ দিয়ে তৈরি ক্যাটাগরিকাল ভেরিয়েবল থেকে? মেশিন লার্নিং মডেলে আমরা যখন কিছু শব্দকে সংখ্যার ক্যাটাগরিতে ভাগ করতে চাই, যেটা এমুহুর্তে সংখ্যায় নেই। যেমন, আইরিশ ডাটাসেটের তিন প্রজাতির ফুলের জন্য ০,১,২, ভাগে ভাগ করতে পারছিনা, সেখানে চলে এসছে এই ডামি ভেরিয়েবল। আমরা যাকে বলছি শব্দকে দিয়ে তার জন্য একটা করে এনকোডিং। এর অর্থ হচ্ছে, আমাদের ভাষায় যতগুলো শব্দ আছে সেগুলোকে ‘ওয়ান হট’ এনকোডিং করা যেতে পারে। আর সেখানেই সমস্যা। 

<img src="https://raw.githubusercontent.com/raqueeb/deep_learning_book/master/assets/hot.png"> চিত্রঃ ‘ওয়ান হট’ এনকোডিং এর উদাহরণ

এখানে একটা উদাহরণ দেয়া যাক। একটা বাক্য। ‘আমি এখন বই পড়ি’। এই চারটা শব্দকে যদি আমরা সংখ্যায় পাল্টাতে চাই, তাহলে আমাদের এই ভোকাবুলারির প্রতিটা শব্দকে ‘০’ ভেক্টর দিয়ে শুরু করব। আমাদের এখানে যেহেতু ৪টা ইউনিক শব্দ, সে কারণে এই শূন্য ভেক্টরের দৈর্ঘ্য হবে ৪। প্রতিটি শব্দকে ঠিকমতো রিপ্রেজেন্ট করার জন্য একেকটা শব্দের ইনডেক্সে তার করেসপন্ডিং ‘১’ বসাবো। ছবি দেখি। এখন এই টেবিল থেকে প্রতিটা শব্দের জন্য তার ভেক্টর বের করা সোজা। আমাদের এই চারটা শব্দের জন্য ভেক্টরের দৈর্ঘ্য হচ্ছে ৪ যার বাকি তিনটাই ০। এভাবে আমরা শব্দকে বিভিন্ন ক্যাটাগরিতে ভাগ করতে পারি। তবে ব্যাপারটা সেরকম এফিশিয়েন্ট নয় যখন আমাদের ভোকাবুলারিতে ১০ হাজার শব্দ থাকবে। তার মানে একেকটা ভেক্টরের দৈর্ঘ্য ১০ হাজার হবে - এর মধ্যে ৯৯.৯৯ শতাংশই হচ্ছে ০। এটা দক্ষ সিস্টেম না। আমরা যদি প্রতিটা বাংলা শব্দের জন্য ডামি ভ্যারিয়েবল বানাই, তাহলে কতো বড় ডামি ভ্যারিয়েবলের একেকটা টেবিল হবে?

এই সমস্যা থেকে বের হবার উপায় কি? প্রতিটা শব্দকে একটা করে ইউনিক সংখ্যা দিয়ে দেওয়া। আমাদের আগের ভোকাবুলারিটাকে ‘আমি এখন বই পড়ি’কে আমরা একটা ‘ডেন্স’ ভেক্টরের মত আলাদা আলাদা করে সংখ্যা দিয়ে দিতে পারি। এই জিনিসটা আগের থেকে অনেকটাই এফিশিয়েন্ট। একেকটা শব্দের জন্য একেকটা আলাদা আলাদা সংখ্যা। সবচেয়ে বড় ব্যাপার হচ্ছে বিশাল ‘স্পার্স’ হাই-ডাইমেনশন স্পেস থেকে কম ডাইমেনশন স্পেসে চলে এলাম। তবে, এটার সমস্যা দুটো।

* ১. আমাদের এই সংখ্যায় এনকোডিং সিস্টেমটাকে ‘ম্যানুয়ালি’ করা হয়েছে। ফলে এদের মধ্যে কোন ‘রিলেশনশিপ’ বের করা যাচ্ছে না। অংকেও এক সংখ্যা থেকে আরেক সংখ্যার মধ্যে যে রিলেশনশিপ সেটা অনুপস্থিত। ফলে আমাদের অংকের ভাষায় কো-সাইন ভেক্টরে কে কোথায় আছে সেটা বের করা মুশকিল।

* ২. একটা মডেলের জন্য এই ধরনের ‘ম্যানুয়াল’ এনকোডিং শব্দগুলোর মধ্যে সম্পর্ক না বুঝলে সেটা সমস্যা হয়ে দাঁড়াবে। যেহেতু এই একেকটা ফিচারের জন্য একেকটা ‘ওয়েট’ সে কারণে একটা লিনিয়ার ক্লাসিফায়ারের পক্ষে এই ফিচার এবং ওয়েট এর কম্বিনেশন কোন সম্পর্ক দেখাবেনা। এটা বড় সমস্যা। 

আর সে কারণেই চলে এসেছে ওয়ার্ড এম্বেডিং।

## একটা ছোট্ট উদাহরন, রেস্টুরেন্ট রিভিউ

আমরা এখানে একটা এমবেডিং লেয়ারের উদাহরণ নিয়ে আসি। ইনপুট ডাইমেনশনে কতগুলো ভোকাবুলারি আছে? ধরে নিচ্ছি ৫০টা। আমি সংখ্যা না গুনেই বলছি। ৫০টা ডামি ভ্যারিয়েবল। মানে আমরা কতগুলো ক্যাটেগরিতে এনকোড করছি। আমাদের লুক আপ টেবিলে কতগুলো আইটেম আছে সেটাই এখানে আসবে। 

আমাদের ৫০টা ভোকাবুলারির জন্য 'ওয়ান হট' এনকোডিং ব্যবহার করছি, কেরাসের ভেতরে। এটা ব্যবহার করা ঠিক না তবে ছোটখাট উদাহরণে সেটা করা যায়। 

In [0]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense

আমরা এখানে ছোট একটা দশটা রেস্টুরেন্টের রিভিউ যোগ করেছি। এর পাশাপাশি তার পাঁচটা নেগেটিভ আর পাঁচটা পজেটিভ লেবেল যোগ করেছি। এর উপর আমরা নিউরাল নেটওয়ার্কে ট্রেইন করব। 

In [0]:
# ১০টা েস্টুরেন্ট রিভিউএর ইনপুট
reviews = [
    'আমি আর আসছি না এখানে!',
    'একদম বাজে সার্ভিস',
    'কথা শোনে না ওয়েটার',
    'একদম ঠান্ডা খাবার',
    'বাজে খাবার!',
    'অসাধারণ',
    'অসাধারণ সার্ভিস!',
    'খুব ভালো!',
    'মোটামুটি',
    'এর থেকে ভালো হয়না']

# লেবেল বলে দিচ্ছি (1=নেগেটিভ, 0=পজেটিভ)
labels = array([1,1,1,1,1,0,0,0,0,0])

In [11]:
print(reviews[0])

আমি আর আসছি না এখানে!


শব্দের সংখ্যা না গুনেও একটা ধারনা নিয়েছি ৫০টা মানে ভোকাবুলারি সাইজ ৫০। ছোট্ট উদাহরণ বলে আমরা ‘ওয়ান হট এনকোডিং’ ব্যবহার করছি। এর কাজ হচ্ছে এই বাক্যগুলোকে শব্দে ভেঙে ফেলা। যাকে আমরা বলছি টোকেনাইজিং। এরপর তাকে নিজস্ব ইনডেক্সে পাঠাবে। শেষে আমরা যা পাব সেটা সংখ্যার সিকোয়েন্স। 

In [12]:
# ওয়ান হট এনকোডিং

VOCAB_SIZE = 50
encoded_reviews = [one_hot(d, VOCAB_SIZE) for d in reviews]
print(f"Encoded reviews: {encoded_reviews}")

Encoded reviews: [[18, 27, 23, 48, 18], [30, 23, 18], [6, 49, 48, 48], [30, 46, 17], [23, 17], [47], [47, 18], [39, 42], [14], [41, 21, 42, 1]]


যতগুলো শব্দ ততগুলো করে সংখ্যা একেকটা অংশে। তবে এই সিকোয়েন্সে একটা কনসিস্টেন্সি রাখার জন্য একটা লেনথ ঠিক করে দিতে হবে। শব্দ ধরে এখন কোনটা ২ অথবা ৩ এবং ৪ লেনথে, কিন্তু প্যাডিং এর জন্য আমরা MAX_LENGTH = 4 ধরে দিচ্ছি। দেখুন এখানে বাকি অংশ ০ দিয়ে ভরে দিয়েছে।

In [13]:
MAX_LENGTH = 4

padded_reviews = pad_sequences(encoded_reviews, maxlen=MAX_LENGTH, padding='post')
print(padded_reviews)

[[27 23 48 18]
 [30 23 18  0]
 [ 6 49 48 48]
 [30 46 17  0]
 [23 17  0  0]
 [47  0  0  0]
 [47 18  0  0]
 [39 42  0  0]
 [14  0  0  0]
 [41 21 42  1]]


আমরা আগের মতো সিকোয়েন্সিয়াল নেটওয়ার্ক তৈরি করছি। আউটপুট ডাইমেনশনে কমিয়ে এনেছি সেটা। এখানে একটা ডেন্স লেয়ার, তার মানে একটা ওয়েট ম্যাট্রিক্স, একটা লার্নিং চলছে এখানে। এই এমবেডিং লেয়ারে যে লার্নিং চলছে সেটা শব্দগুলোকে একটা ইউক্লুডিয়ান স্পেসে সংখ্যাগুলোর মধ্যে একটা রিলেশনশিপ তৈরি হচ্ছে।

মডেলটা দেখুন।

In [14]:
model = Sequential()
embedding_layer = Embedding(VOCAB_SIZE, 8, input_length=MAX_LENGTH)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
# মডেলকে ট্রেইন করি
model.fit(padded_reviews, labels, epochs=100, verbose=0)

১০০ ইপকের পর ভালো করে লক্ষ্য করুন, প্রতিটা লাইন হচ্ছে একেকটা শব্দের এমবেডিং। এখানে ওয়েটগুলোর অর্থ এখন খুঁজতে যাবো না এমুহুর্তে।

In [16]:
print(embedding_layer.get_weights()[0].shape)
print(embedding_layer.get_weights())

(50, 8)
[array([[ 0.08445652, -0.09346382, -0.1153101 , -0.08687168,  0.12489384,
         0.06159101, -0.15283671,  0.16201596],
       [ 0.08379133, -0.11405947, -0.14295566, -0.10899172,  0.14112172,
        -0.10753997,  0.05359931,  0.12398995],
       [-0.00186111,  0.00924151,  0.03309018, -0.0285947 , -0.01509882,
        -0.04974305,  0.00719942,  0.0233702 ],
       [ 0.02450119, -0.03862574, -0.01956166,  0.01122971, -0.02732921,
        -0.01081762, -0.04372646, -0.03618447],
       [ 0.03053823, -0.01402362,  0.01821253, -0.01614489,  0.04108104,
        -0.01281965, -0.02603277, -0.01150078],
       [ 0.04556557,  0.01202744, -0.04501692, -0.00029296,  0.02726028,
         0.04015929,  0.00935964, -0.0383476 ],
       [-0.13580176, -0.09986364,  0.1245888 ,  0.10950329, -0.05969741,
        -0.13374738,  0.11801434,  0.07084785],
       [-0.03462787, -0.04161174, -0.00193647, -0.01973833,  0.00555579,
        -0.0073867 ,  0.04086855,  0.03668931],
       [ 0.03034202, -0

অ্যাক্যুরেসি কেন ১ এসেছে? কারণ, দশটা বাক্যের মধ্যে সবগুলোই ইউনিক। একটার সাথে আরেকটার মিল নেই। আর, এতো ছোট উদাহরণে আর কি হবে?

In [17]:
loss, accuracy = model.evaluate(padded_reviews, labels, verbose=0)
print(f'Accuracy: {accuracy}')

Accuracy: 1.0


আরেকটা উদাহরণ দেখি। এসেছে স্ট্যাকওভারফ্লো থেকে।

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
import numpy as np

এখানেও আমরা একটা বাক্যকে ক্লাসিফাই করবো। 'ব্যাগ অফ ওয়ার্ড' (একটা ব্যাগে যতো শব্দ আছে) হচ্ছে একটা মেকানিজম যার মাধ্যমে টেক্সট থেকে ফিচার বের করা যায়। এটা আসলে আমাদের এই তিনটা উদাহরন থেকে যতো ইউনিক শব্দ আছে সেটা থেকে সে ট্রেনিং নেয়। তবে, আমরা তার থেকেও ভালো মডেল মানে সিকোয়েন্সের ওপর জোর দেবো।

In [19]:
num_classes = 5 
max_words = 20
sentences = ['আমি আর আসছি না এখানে!',
              'কথা শোনে না ওয়েটার',
            'একদম ঠান্ডা খাবার']
labels = np.random.randint(0, num_classes, 3)
y = to_categorical(labels, num_classes=num_classes)

words = set(w for sent in sentences for w in sent.split())
word_map = {w : i+1 for (i, w) in enumerate(words)}
sent_ints = [[word_map[w] for w in sent.split()] for sent in sentences]
vocab_size = len(words)
print(vocab_size)

11


বাক্যের শব্দের সিকোয়েন্স, ৫, ৪, ৩

In [22]:
print(sent_ints)

[[7, 3, 2, 6, 10], [5, 11, 6, 1], [8, 9, 4]]


আমাদেরকে প্যাড করতে হবে max_words লেনথ ধরে, যা পাবো len(words) এবং ১ যোগ করে। +১ মানে হচ্ছে আমরা ০কে রিজার্ভ রাখবো যাতে সেটার বাইরে না যায়। এখানে ওয়ান হট এনকোডিং এবং প্যাডিং করছি -

In [23]:
X = np.array([to_categorical((pad_sequences((sent,), 
    max_words)).reshape(20,),vocab_size + 1) for sent in sent_ints])
print(X.shape)

(3, 20, 12)


এখন দেখি আমাদের ওয়ান হট এনকোডিং এর অবস্থা?

In [24]:
print(X)

[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]

 [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [27]:
print(y)

[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


মডেলে ডেন্স লেয়ার যোগ করছি যাতে ওয়ান হট শব্দগুলোকে ডেন্স ভেক্টরে পাল্টে নেয়া যায়। তো LSTM এর কাজ কি? আমাদের বাক্যের ভেতরে শব্দের ভেক্টরকে কনভার্ট করতে হবে ডেন্স বাক্য ভেক্টরে। একদম শেষ লাইনে সফটম্যাক্স এক্টিভেশন ফাংশন ব্যবহার করছি যাতে ক্লাসের ওপর প্রবাবিলিটি ডিস্ট্রিবিউশন চালাতে পারে। 

In [0]:
model = Sequential()
model.add(Dense(512, input_shape=(max_words, vocab_size + 1)))
model.add(LSTM(128))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

In [26]:
model.fit(X,y)

Train on 3 samples
3/3 [==============================] - 4s 1s/sample - loss: 1.6413 - accuracy: 0.0000e+00


## শব্দ এবং সংখ্যার কাছাকাছি এনকোডিং

শব্দ যখন সংখ্যা হবে, তখন শব্দের মধ্যে যেরকম সম্পর্ক ছিলো, সেটা আরো ভালো বোঝা যাবে সংখ্যায়। শব্দে "আমি" "তুমি" যেমন কাছাকাছি, সংখ্যায় সেটা আরো পরিস্কার হবে। এমবেডিং এ এটা এমন ধরনের ‘ডেন্স’ রিপ্রেজেন্টেশন যার মধ্যে একই ধরনের শব্দের একই বা কাছাকাছি ধরনের এনকোডিং হবে। ‘পুরুষ’ এবং ‘মহিলা’ এই দুটো অক্ষরের মধ্যে আকাশ পাতাল পার্থক্য হলেও সম্পর্কের কারণে এদুটো কাছাকাছি থাকবে। সংখ্যার এনকোডিং ও একই ধরনের হবে। সবচেয়ে বড় কথা হচ্ছে এ ধরনের এনকোডিং ম্যানুয়ালি বা আমাদের হাতে লিখে দিতে হবে না। বরং যেহেতু এগুলো ট্রেনিংযোগ্য প্যারামিটার ফলে মডেলের ট্রেনিং এর সময় ওয়েটগুলো ‘এডজাস্ট’ হবে এদের সম্পর্কের ভিত্তিতে। একটা ছোট ডাটাসেটের ওয়ার্ড এম্বেডিং ৮ ডাইমেনশনাল হলেও সেটা বড় ডাটাসেটের জন্য ১০২৪ ডাইমেনশন পর্যন্ত যেতে পারে। যত বেশি ডাইমেনশনাল এম্বেডিং ততবেশি শব্দগুলোর মধ্যে সম্পর্কগুলোকে আরো ভালোভাবে বোঝা যাবে তবে তার জন্য প্রচুর ডাটা লাগবে শিখতে।

আমরা যখন ‘ওয়ান হট’ এনকোডিং উদাহরণ দেখছিলাম, সেখানে প্রতিটা শব্দ একটা ৪ ডাইমেনশনাল ফ্লোটিং পয়েন্ট সংখ্যা ভেক্টর দিয়ে রিপ্রেজেন্ট করা হয়েছিল। আমরা এ ধরনের এম্বেডিংকে বলতে পারি ‘লুকআপ’ টেবিল। এই ‘লুকআপ’ টেবিলের ওয়েটগুলো শিখছে যখন আমরা ‘ডেন্স’ ভেক্টরের তার করেসপন্ডিং টেবিল দেখছি। ফলে সেটা ধরে আমরা প্রতিটা শব্দ ধরে এনকোডিং করছি। তবে, এখানে একটা বড় সমস্যা আছে। আমরা যখন এনকোডিং করবো তখন সব বাংলা শব্দকে একসাথে এনকোডিং না করলে আমরা কিভাবে একটা শব্দকে আরেকটা শব্দের সাথে সিমিলারিটি, শব্দগুলোর মধ্যে দূরত্ব বের করবো? সে সমস্যা মেটাতে এসেছে প্রি-ট্রেইনড মডেল। কোটি কোটি শব্দকে একসাথে ট্রেইন করিয়ে তারপর এনকোডিং করেছে একসাথে। ফলে এধরনের প্রি-ট্রেইনড মডেলগুলো অসাধারণ পারদর্শী। শুধু নামিয়ে নিতে হবে দরকারের সময়ে।

ধারণাগুলো এসেছে জেফ হিটনের নোটবুক এবং স্ট্যাকওভারফ্লো থেকে।

https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_11_03_embedding.ipynb